In [1]:
import glob
import pandas as pd
import numpy as np
import pickle
import re
from bs4 import BeautifulSoup
import collections
import requests
from itertools import chain
from tqdm import tqdm
import string

In [2]:
# Init
agro_list = []

for letter in tqdm(list(string.ascii_uppercase)):
    # Get URL
    URL = 'http://apps.worldagroforestry.org/treedb2/index.php?letter={0}'.format(letter)
    page = requests.get(URL, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Extract search table
    for tree in soup.find_all('table')[2].find_all('a'):
        agro_list.append(tree.text)

100%|███████████████████████████████████████████| 26/26 [00:09<00:00,  2.78it/s]


In [3]:
agro_urls = ['http://db.worldagroforestry.org//species/properties/' + '_'.join(tree.split(' ')) for tree in agro_list]

agro_data = collections.defaultdict(list)

# Loop over URL
for URL, name in zip(tqdm(agro_urls), agro_list):
    page = requests.get(URL, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    for text in soup.find_all('div'):
        try:
            if text.find_previous_sibling().text.strip() == 'Botanic':
                agro_data[name].append(tuple([1, text.text.strip()]))
            else:
                agro_data[name].append(tuple([0, text.text.strip()]))
        except:
            continue

100%|█████████████████████████████████████████| 617/617 [05:44<00:00,  1.79it/s]


In [4]:
with open('../../data/processed/descriptions_web_agroforestry.pkl', 'wb') as f:
    pickle.dump(agro_data, f)    